In [1]:
import sys
sys.path.append('..')

First, we import the required libraries and methods:

In [2]:
import pandas as pd
from tqdm import tqdm
from faker import Faker

from utils.providers.NIFProvider import NIFProvider

- ``pandas``: Data generated with Faker will be stored in a Pandas DataFrame before being imported into the database.
- ``tqdm()``: This method is required for adding a progress bar to show the progress of the DataFrame creation.
- ``Faker()``: It’s the generator from the faker library.
- ``cpu_count()``: This is a method from the multiprocessing module that will return the number of cores available.

We also import extra providers needed. For info on them, check their documentation.

Then, a faker generator will be created and initialized, by calling the ``Faker()`` method with the following arguments:
- ``locale`` is set to "pt_PT", for generating Portuguese data.

And we determine the number of cores of the CPU available, by calling the ``cpu_count()`` method and assigning this value to the ``num_cores`` variable. We use all the cores minus one to avoid freezing the computer.

Finally, we also add the needed providers imported earlier to faker.

In [3]:
fake = Faker(locale='pt_PT')
fake.add_provider(NIFProvider)

In [4]:
phone_formats =['91#######', '92#######', '93#######', '2########']
life_ensurance_format = ['#########', '']

Then, we define the ``create_dataframe()`` function:

In [5]:
def create_dataframe(x):
    data = pd.DataFrame()
    for i in tqdm(range(x), desc='Creating DataFrame'):
        data.loc[i, 'Nome'] = fake.name()
        data.loc[i, 'NIF'] = fake.nif()
        data.loc[i, 'Salario'] = f"{fake.random.randint(1000, 999999)}.{fake.random.randint(0, 99)}"
        data.loc[i, 'SeguroVida'] = fake.numerify(fake.random_element(life_ensurance_format))
        data.loc[i, 'Email'] = fake.email(domain='veritatis.pt')
        data.loc[i, 'Telefone'] = fake.numerify(fake.random_element(phone_formats))
    return data

Here, ``x`` is the variable that will determine the number of iterations of the for loop where the ``DataFrame`` is created.
data is an empty ``DataFrame`` that will later be fulfilled with data generated with ``Faker``.
Pandas ``DataFrame.loc`` attribute provides access to a group of rows and columns by their label(s). In each iteration, a row of data is added to the ``DataFrame`` and this attribute allows assigning values to each column.

The ``DataFrame`` that is created after calling this function will have the following columns:

```
 #   Column      Dtype   Null      Count
---  ----------  ------  --------  -----
 0   Nome        object  non-null  100
 2   NIF         object  non-null  100
 3   Salario     object  non-null  100
 4   SeguroVida  object  null      100
 5   Email       object  non-null  100
 6   Telefone    object  null      100
```

**Note**: The script is generating 100 records, you can modify this value in the with the argument ``x`` variable.

In [6]:
data = create_dataframe(100) # edit `x` here
data.to_csv('../data/generated_employees.csv', index=True, index_label='Id')

Creating DataFrame: 100%|██████████| 100/100 [00:00<00:00, 1765.98it/s]
